<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-Libraries-and-declaring-global-variables" data-toc-modified-id="Importing-Libraries-and-declaring-global-variables-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing Libraries and declaring global variables</a></span></li><li><span><a href="#Creating-dataset" data-toc-modified-id="Creating-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating dataset</a></span></li><li><span><a href="#Calculating-relative-support-condition" data-toc-modified-id="Calculating-relative-support-condition-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Calculating relative support condition</a></span></li><li><span><a href="#Defining-functions-for-pruning-and-joining" data-toc-modified-id="Defining-functions-for-pruning-and-joining-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Defining functions for pruning and joining</a></span></li><li><span><a href="#Recursive-function-for-finding-candidates-and-minimum-support" data-toc-modified-id="Recursive-function-for-finding-candidates-and-minimum-support-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Recursive function for finding candidates and minimum support</a></span></li><li><span><a href="#Calculating-confidence" data-toc-modified-id="Calculating-confidence-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Calculating confidence</a></span></li><li><span><a href="#Association-rules-generated" data-toc-modified-id="Association-rules-generated-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Association rules generated</a></span></li><li><span><a href="#Strong-rules" data-toc-modified-id="Strong-rules-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Strong rules</a></span></li><li><span><a href="#Weak-rules" data-toc-modified-id="Weak-rules-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Weak rules</a></span></li></ul></div>

# Importing Libraries and declaring global variables

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
import copy

In [2]:
pd.options.mode.chained_assignment = None
global support_condition
global sup_count
global save_df 

# Creating dataset

In [3]:
data = {'TID': ['T100','T200','T300','T400','T500','T600','T700','T800','T900'],
        'I1' : [1, 0, 0, 1, 1, 0, 1, 1, 1],
        'I2' : [1, 1, 1, 1, 0, 1, 0, 1, 1],
        'I3' : [0, 0, 1, 0, 1, 1, 1, 1, 1],
        'I4' : [0, 1, 0, 1, 0, 0, 0, 0, 0],
        'I5' : [1, 0, 0, 0, 0, 0, 0, 1, 0]
       }
df = pd.DataFrame(data, columns = ['TID', 'I1', 'I2', 'I3', 'I4', 'I5'])
columns = ['I1', 'I2', 'I3', 'I4', 'I5']
df.set_index('TID', inplace = True)
count = 1
save_df = []
c1 = 3

In [4]:
display(df)

,I1,I2,I3,I4,I5
TID,,,,,
T100,1,1,0,0,1
T200,0,1,0,1,0
T300,0,1,1,0,0
T400,1,1,0,1,0
T500,1,0,1,0,0
T600,0,1,1,0,0
T700,1,0,1,0,0
T800,1,1,1,0,1
T900,1,1,1,0,0


# Calculating relative support condition

In [5]:
support_condition = 2
rows = df.shape[0]
relative_support = support_condition/rows
print('Relative support : ',round(relative_support,2))
df1 = {}

Relative support :  0.22


# Defining functions for pruning and joining

In [6]:
def calc_support_count(itemset, count):
    if count == 1:
        sup_count = []
        for i in itemset:
            temp = df[i]
            temp = temp.sum(axis = 0)
            sup_count.append(temp)
            count = count + 1
        return sup_count
    else:
        ind = df.index.values.tolist()
        l = len(itemset[0])
        #print(l)
        val = []
        #print(l)
        for i in itemset:
            s2 = 0
            for j in ind:
                s1 = 0
                for k in i:
                    #print(i,j,k)
                    #print(' {} {} : {}'.format(j, k, df.loc[j,k]))
                    s1 = s1 + df.loc[j,k]
                #print(s1)
                if s1 == l:
                    s2 = s2 + 1
                    #print('Yes')
                else:
                    #print('No')
                    pass
            val.append(s2)
            count = count + 1
        return val

In [7]:
def min_support(rt, val):
    temp = []
    for i in rt:
        s = ''
        for j in i:
            #print(j)
            s = s + j
        temp.append(s)
    d = { k:v for (k,v) in zip(temp, val)}
    new_df = pd.DataFrame(list(zip(temp, val)), columns =['Itemset', 'Values'])
    sup_df = new_df[new_df['Values'] >= 2]
    return new_df, sup_df

# Recursive function for finding candidates and minimum support

In [8]:
def func(columns):
    global count
    global df1
    #display(df1)
    val = calc_support_count(columns, count)
    new_df, sup_df = min_support(columns, val)
    save_df.append(sup_df)
    count = count + 1
    if new_df.empty or sup_df.empty:
        return df1
    else:
        print('Candidates (C{}) : '.format(count-1))
        new_df.set_index('Itemset', inplace = True)
        display(new_df)
        new_df.reset_index(inplace = True)
        print('\n Candidates greater than minimum support count (L{}): '.format(count-1))
        sup_df.set_index('Itemset', inplace = True)
        display(sup_df)
        sup_df.reset_index(inplace = True)
        #print(count)
        if count > 1:
            new_items = df.columns.values.tolist()
            new_items = list(combinations(new_items, count))
        else:
            pass
        func(new_items)

In [9]:
sup_df = func(columns)

Candidates (C1) : 


,Values
Itemset,
I1,6
I2,7
I3,6
I4,2
I5,2



 Candidates greater than minimum support count (L1): 


,Values
Itemset,
I1,6
I2,7
I3,6
I4,2
I5,2


Candidates (C2) : 


...

,Values
Itemset,
I1I2,4
I1I3,4
I1I5,2
I2I3,4
I2I4,2
I2I5,2


Candidates (C3) : 


,Values
Itemset,
I1I2I3,2
I1I2I4,1
I1I2I5,2
I1I3I4,0
I1I3I5,1
I1I4I5,0
I2I3I4,0
I2I3I5,1
I2I4I5,0



 Candidates greater than minimum support count (L3): 


,Values
Itemset,
I1I2I3,2
I1I2I5,2


# Calculating confidence

In [10]:
final_df = save_df[len(save_df) - 2]
items = final_df['Itemset'].values.tolist()
l = len(items[0])
l = int(l/c1)
unique_values = [[] for i in range(l)]
s = ''
c = -1
for i in items:
    c = c + 1
    for j in range(0, len(i), 2):
        s = i[j] + i[j+1]
        unique_values[c].append(s)

In [11]:
l1 = []
combi = list(combinations(unique_values[0], 2))
print(combi)
for i in combi:
    s = ''
    for j in i:
        s = s + str(j)
    l1.append(s)
    
l2 = []
combi = list(combinations(unique_values[1], 2))
for i in combi:
    s = ''
    for j in i:
        s = s + str(j)
    l2.append(s)

[('I1', 'I2'), ('I1', 'I3'), ('I2', 'I3')]


In [12]:
d_df = {}
c = 0
for i in save_df:
    c = c = c + 1
    d_df[c] = i

In [13]:
df1 = save_df[0]
cols_1 = df1['Itemset'].values.tolist()
df1.set_index('Itemset', inplace = True)
df2 = save_df[1]
cols_2 = df2['Itemset'].values.tolist()
df2.set_index('Itemset', inplace = True)

In [14]:
c1 = {}
c2 = {}
final_df.set_index('Itemset', inplace = True)
for i in l1:
    c1[i] = (final_df.loc['I1I2I3', 'Values']/ df2.loc[i, 'Values'])*100
for i in unique_values[0]:
    c1[i] = (final_df.loc['I1I2I3', 'Values']/ df1.loc[i, 'Values'])*100
for i in l2:
    c2[i] = (final_df.loc['I1I2I3', 'Values']/ df2.loc[i, 'Values'])*100
for i in unique_values[1]:
    c2[i] = (final_df.loc['I1I2I3', 'Values']/ df1.loc[i, 'Values'])*100

# Association rules generated

In [15]:
for keys, values in c1.items():
    print('Confidence for {} = {}%'.format(keys,round(values, 2)))
for keys, values in c2.items():
    print('Confidence for {} = {}%'.format(keys,round(values, 2)))

Confidence for I1I2 = 50.0%
Confidence for I1I3 = 50.0%
Confidence for I2I3 = 50.0%
Confidence for I1 = 33.33%
Confidence for I2 = 28.57%
Confidence for I3 = 33.33%
Confidence for I1I2 = 50.0%
Confidence for I1I5 = 100.0%
Confidence for I2I5 = 100.0%
Confidence for I1 = 33.33%
Confidence for I2 = 28.57%
Confidence for I5 = 100.0%


In [16]:
min_condidence = 70.0
d_strong = {}
d_weak = {}
for keys, values in c1.items():
    if values >= min_condidence:
        d_strong[keys] = values
    else:
        d_weak[keys] = values
for keys, values in c2.items():
    if values >= min_condidence:
        d_strong[keys] = values
    else:
        d_weak[keys] = values

# Strong rules

In [17]:
print('Minimum confidence = 70% \n')
print('Strong rules : ')
for keys, values in d_strong.items():
    print('Confidence for {} = {}%'.format(keys,round(values, 2)))

Minimum confidence = 70% 

Strong rules : 
Confidence for I1I5 = 100.0%
Confidence for I2I5 = 100.0%
Confidence for I5 = 100.0%


# Weak rules

In [18]:
print('Minimum confidence = 70% \n')
print('Weak rules : ')
for keys, values in d_weak.items():
    print('Confidence for {} = {}%'.format(keys,round(values, 2)))

Minimum confidence = 70% 

Weak rules : 
Confidence for I1I2 = 50.0%
Confidence for I1I3 = 50.0%
Confidence for I2I3 = 50.0%
Confidence for I1 = 33.33%
Confidence for I2 = 28.57%
Confidence for I3 = 33.33%
